In [1]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

/var/folders/1j/mpxdgh255lj4_vy0kyxxrx200000gn/T/ipykernel_10597/31679372.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [131]:
psn_df = pd.read_excel('/Volumes/LaCie/event_ews/old_psn.xlsx')
#asrs_df = pd.read_csv('/Volumes/LaCie/event_ews/cmb_asrs.csv')

In [132]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
psn_df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in psn_df['Narrative_merged']]

#asrs_df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in asrs_df['report_1narrative']]

In [133]:
psn_df['embeddings'] = psn_df['Narrative_merged'].map(lambda x: model.encode(x, convert_to_tensor=False))
#asrs_df['embeddings'] = asrs_df['report_1narrative'].map(lambda x: model.encode(x,convert_to_tensor=False))

In [143]:
psn_df.rename(columns={'FLR_SUBMIT_DATE':'date'},inplace=True)

In [113]:
def group_sim_mean(x):
    sim = cosine_similarity(list(x))
    np.fill_diagonal(sim,np.nan)
    return sim[~np.isnan(sim)].mean()

def group_sim_std(x):
    sim = cosine_similarity(list(x))
    np.fill_diagonal(sim,np.nan)
    return sim[~np.isnan(sim)].std()

In [174]:
psn_vader_metrics = psn_df[['PRIMARY_LOC_NAME','date','compound']]
psn_vader_metrics = psn_vader_metrics.groupby(['PRIMARY_LOC_NAME',pd.Grouper(key='date', freq='D')]).agg(['mean','std']).reset_index()
#psn_vader_metrics.columns = ['date','vader_comp_mean','vader_comp_std']
psn_vader_metrics.columns = psn_vader_metrics.columns.map('_'.join).str.strip('_')
psn_vader_metrics.head()

,PRIMARY_LOC_NAME,date,compound_mean,compound_std
0,1-Not Applicable,2013-07-08,0.888500,NaN
1,1-Not Applicable,2013-07-15,-0.246133,0.954765
2,1-Not Applicable,2013-07-16,0.540200,NaN
3,1-Not Applicable,2013-07-24,0.822500,NaN
4,1-Not Applicable,2013-08-05,-0.431000,NaN


In [185]:
#psn_sim_metrics = psn_df[['date','embeddings']]
psn_sim_metrics = psn_df[['PRIMARY_LOC_NAME','date','embeddings']]
#psn_sim_metrics['date']
#psn_sim_metrics = psn_sim_metrics.groupby(pd.Grouper(key='date', freq='D')).apply(np.mean).reset_index(name='mean_embedding')
psn_sim_metrics = psn_sim_metrics.groupby(['PRIMARY_LOC_NAME',pd.Grouper(key='date', freq='D')]).apply(np.mean).reset_index(name='mean_embedding')
#psn_sim_metrics['mean_embedding_shift'] = psn_sim_metrics['mean_embedding'].shift(periods=1)
psn_sim_metrics['mean_embedding_shift'] = psn_sim_metrics.groupby('PRIMARY_LOC_NAME')['mean_embedding'].shift(periods=1)
psn_sim_metrics.dropna(axis = 0,inplace=True)
#psn_sim_metrics['lag_sim'] = psn_sim_metrics.apply(lambda x: cosine_similarity([x.mean_embedding,x.mean_embedding_shift])[0,1], axis=1)
psn_sim_metrics['lag_sim'] = psn_sim_metrics.apply(lambda x: cosine_similarity([x.mean_embedding,x.mean_embedding_shift])[0,1], axis = 1)
psn_sim_metrics.head()

,PRIMARY_LOC_NAME,date,mean_embedding,mean_embedding_shift,lag_sim
1,1-Not Applicable,2013-07-15,"[-0.028988875, 0.025440848, 0.0107028065, 0.03...","[-0.057235606, 0.10142736, 0.014249456, 0.0303...",0.543528
2,1-Not Applicable,2013-07-16,"[-0.083788216, 0.020813072, 0.10173995, -0.006...","[-0.028988875, 0.025440848, 0.0107028065, 0.03...",0.350543
3,1-Not Applicable,2013-07-24,"[-0.013506578, -0.04957799, 0.02950874, -0.028...","[-0.083788216, 0.020813072, 0.10173995, -0.006...",0.142326
4,1-Not Applicable,2013-08-05,"[0.07154466, 0.049656373, 0.011825556, 0.08592...","[-0.013506578, -0.04957799, 0.02950874, -0.028...",0.317988
5,1-Not Applicable,2013-08-06,"[-0.066216335, -0.0047464226, -0.008616552, 0....","[0.07154466, 0.049656373, 0.011825556, 0.08592...",0.305237


In [214]:
psn_sim_metrics2 = psn_df[['PRIMARY_LOC_NAME','date','embeddings']]
psn_sim_metrics2 = psn_sim_metrics2.groupby(['PRIMARY_LOC_NAME',pd.Grouper(key='date', freq='D')]).apply(np.mean).reset_index(name='mean_embedding')
psn_sim_metrics2 = psn_sim_metrics2.groupby('PRIMARY_LOC_NAME').filter(lambda x: len(x) >120)
psn_sim_metrics2.dropna(axis = 0,inplace=True)
psn_sim_metrics2.head()
#psn_sim_metrics2['rolling_sim'] = psn_sim_metrics2.groupby('PRIMARY_LOC_NAME')['mean_embedding'].apply(lambda g: g.rolling(5,min_periods=1,center=False).apply(group_sim_mean))
psn_sim_metrics2['rolling_sim'] = psn_sim_metrics2.rolling(2, on = 'date',min_periods=1)['mean_embedding'].apply(group_sim_mean).reset_index(0,drop=True)

DataError: No numeric types to aggregate

In [206]:
len(psn_sim_metrics2)

4405

In [84]:
asrs_sim_metrics = asrs_df[['date','embeddings']]
asrs_sim_metrics = asrs_sim_metrics.groupby('date').apply(np.mean).reset_index(name='mean_embedding')
asrs_sim_metrics['mean_embedding_shift'] = asrs_sim_metrics['mean_embedding'].shift(periods=1)
asrs_sim_metrics.dropna(axis = 0,inplace=True)
asrs_sim_metrics['lag_sim'] = asrs_sim_metrics.apply(lambda x: cosine_similarity([x.mean_embedding,x.mean_embedding_shift])[0,1], axis=1)
asrs_sim_metrics.head()

,date,mean_embedding,mean_embedding_shift,lag_sim
1,1988-02-01,"[-0.061027892, -0.028464677, 0.015833795, 0.01...","[-0.05780552, -0.01677651, 0.022315733, 0.0217...",0.992453
2,1988-03-01,"[-0.058345165, -0.025108237, 0.014209407, 0.01...","[-0.061027892, -0.028464677, 0.015833795, 0.01...",0.995864
3,1988-04-01,"[-0.05758096, -0.023219632, 0.014241583, 0.017...","[-0.058345165, -0.025108237, 0.014209407, 0.01...",0.996180
4,1988-05-01,"[-0.05582636, -0.022245584, 0.01852573, 0.0172...","[-0.05758096, -0.023219632, 0.014241583, 0.017...",0.995060
5,1988-06-01,"[-0.057438016, -0.026060024, 0.012747915, 0.01...","[-0.05582636, -0.022245584, 0.01852573, 0.0172...",0.994891


In [123]:
asrs_sim_metrics2 = asrs_df[['date','embeddings']]
asrs_sim_metrics2 = asrs_sim_metrics2.groupby('date')['embeddings'].apply(group_sim_mean).reset_index(name='mean_sim')
asrs_sim_metrics3 = asrs_df[['date','embeddings']]
asrs_sim_metrics3 = asrs_sim_metrics3.groupby('date')['embeddings'].apply(group_sim_std).reset_index(name='std_sim')

/var/folders/v6/ztvgzd816jd95zsvrnl_zb2c0000gp/T/ipykernel_97939/2835754924.py:4: RuntimeWarning: Mean of empty slice.
  return sim[~np.isnan(sim)].mean()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


In [165]:
psn_sim_metrics2 = psn_df[['date','embeddings']]
psn_sim_metrics2 = psn_sim_metrics2.groupby(pd.Grouper(key='date', freq='D'))['embeddings'].apply(group_sim_mean).reset_index(name='mean_sim')
psn_sim_metrics3 = psn_df[['date','embeddings']]
psn_sim_metrics3 = psn_sim_metrics3.groupby(pd.Grouper(key='date', freq='D'))['embeddings'].apply(group_sim_std).reset_index(name='std_sim')

In [166]:
all_psn_metrics = psn_sim_metrics[['date','lag_sim']].set_index('date').join(psn_sim_metrics2.set_index('date'))
all_psn_metrics = all_psn_metrics.join(psn_sim_metrics3.set_index('date'))
all_psn_metrics = all_psn_metrics.join(psn_vader_metrics.set_index('date'))
all_psn_metrics.head()

,lag_sim,mean_sim,std_sim,vader_comp_mean,vader_comp_std
date,,,,,
2013-07-02,0.888988,0.272151,0.109881,-0.072512,0.508136
2013-07-03,0.934109,0.290662,0.116754,-0.172340,0.520796
2013-07-04,0.870342,0.327987,0.097423,-0.304673,0.492164
2013-07-05,0.846638,0.298775,0.120106,-0.265121,0.534005
2013-07-06,0.867586,0.276458,0.107107,-0.027556,0.367412


In [167]:
all_psn_metrics.to_csv('/Volumes/LaCie/event_ews/all_psn_metrics_12-28-2023.csv')

In [129]:
all_asrs_metrics = asrs_sim_metrics[['date','lag_sim']].set_index('date').join(asrs_sim_metrics2.set_index('date'))
all_asrs_metrics = all_asrs_metrics.join(asrs_sim_metrics3.set_index('date'))
all_asrs_metrics = all_asrs_metrics.join(vader_metrics.set_index('date'))
all_asrs_metrics.head()

,lag_sim,mean_sim,std_sim,vader_comp_mean,vader_comp_std
date,,,,,
1988-02-01,0.992453,0.422769,0.116777,-0.319939,0.686014
1988-03-01,0.995864,0.432620,0.113538,-0.344239,0.673899
1988-04-01,0.996180,0.435843,0.110778,-0.377918,0.652923
1988-05-01,0.995060,0.429069,0.109929,-0.358967,0.656970
1988-06-01,0.994891,0.428424,0.111934,-0.350056,0.648810


In [130]:
all_asrs_metrics.to_csv('/Volumes/LaCie/event_ews/all_asrs_metrics_12-28-2023.csv')